# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [15]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [16]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [17]:
spaceship = spaceship.dropna()

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [18]:
#your code here
features = spaceship.drop(columns = ["Transported", "PassengerId", "Name"])
target = spaceship["Transported"]

In [19]:
spaceship["Cabin"].unique()

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      dtype=object)

In [20]:
spaceship['Cabin'].apply(lambda x: isinstance(x, float)).any()

False

In [21]:
spaceship = spaceship[spaceship['Cabin'].apply(lambda x: not isinstance(x, float))]

In [22]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x.split('/')[0])
spaceship['Cabin'].unique()

array(['B', 'F', 'A', 'G', 'E', 'C', 'D', 'T'], dtype=object)

In [23]:
spaceship.drop(columns=['PassengerId', 'Name'], inplace=True)

In [24]:
spaceship = pd.get_dummies(spaceship, columns=['Cabin', 'HomePlanet', 'Destination', 'VIP', 'CryoSleep'])
spaceship.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_A,Cabin_B,Cabin_C,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,CryoSleep_False,CryoSleep_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,...,False,True,False,False,False,True,True,False,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,...,True,False,False,False,False,True,True,False,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,...,False,True,False,False,False,True,False,True,True,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,...,False,True,False,False,False,True,True,False,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,...,True,False,False,False,False,True,True,False,True,False


In [25]:
features = spaceship.drop(columns = ["Transported"])
target = spaceship["Transported"]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [27]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [28]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [29]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,CryoSleep_False,CryoSleep_True
0,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [30]:
#your code here
from sklearn.ensemble import GradientBoostingClassifier

# Crear el modelo GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(max_depth=15,
                                    n_estimators=100)

In [31]:
gb_clf.fit(X_train_norm, y_train)

GradientBoostingClassifier(max_depth=15)

- Evaluate your model

In [32]:
#your code here
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
pred = gb_clf.predict(X_test_norm)

# Calcular y mostrar las métricas de clasificación
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, average='weighted'))
print("Recall:", recall_score(y_test, pred, average='weighted'))
print("F1 Score:", f1_score(y_test, pred, average='weighted'))

# Matriz de confusión (opcional)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7980332829046899
Precision: 0.7980339650279227
Recall: 0.7980332829046899
F1 Score: 0.7980331673421325
Confusion Matrix:
 [[528 133]
 [134 527]]


c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [34]:
#your code here
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV

grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [35]:
# Crear el modelo AdaBoostClassifier con DecisionTreeClassifier 
bagging_clf = AdaBoostClassifier(DecisionTreeClassifier())

In [36]:
# Crear un GridSearchCV para encontrar los mejores parámetros 
model = GridSearchCV(bagging_clf, param_grid=grid, cv=5)

- Run Grid Search

In [37]:
# Ajustar el GridSearchCV a tus datos de entrenamiento 
model.fit(X_train_norm, y_train)

c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\bru

GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

- Evaluate your model

In [38]:
model.best_params_

{'estimator__max_depth': 10,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 50}

# (BONUS) Randon Search

In [39]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
        "estimator__max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
        "estimator__max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

In [40]:
grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'estimator__max_leaf_nodes': [500,
  777,
  1055,
  1333,
  1611,
  1888,
  2166,
  2444,
  2722,
  3000],
 'estimator__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110]}

In [41]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier())

model = RandomizedSearchCV(estimator = ada_clf, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)

In [42]:
model.fit(X_train_norm,y_train)

c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier()),
                   n_jobs=-1,
                   param_distributions={'estimator__max_depth': [10, 20, 30, 40,
                                                                 50, 60, 70, 80,
                                                                 90, 100, 110],
                                        'estimator__max_leaf_nodes': [500, 777,
                                                                      1055,
                                                                      1333,
                                                                      1611,
                                                                      1888,
                                                                      2166,
                                                                      2444,
                                                                      2722,
                                                                      3000],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [43]:
model.best_params_

{'n_estimators': 600,
 'estimator__max_leaf_nodes': 1888,
 'estimator__max_depth': 20}

In [44]:
best_model = model.best_estimator_

In [ ]:
help(model)

In [46]:
pred = gb_clf.predict(X_test_norm)

# Calcular y mostrar las métricas de clasificación
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, average='weighted'))
print("Recall:", recall_score(y_test, pred, average='weighted'))
print("F1 Score:", f1_score(y_test, pred, average='weighted'))

# Matriz de confusión (opcional)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7980332829046899
Precision: 0.7980339650279227
Recall: 0.7980332829046899
F1 Score: 0.7980331673421325
Confusion Matrix:
 [[528 133]
 [134 527]]


c:\Users\bruno\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
